In [ ]:
# Notebook to test the Bedrock Agents and UC Functions.

In [20]:
import boto3, pprint, json, time, uuid

In [12]:
# Create Boto3 client for IAM and STS

iam_client = boto3.client('iam')
sts_client = boto3.client('sts')

In [ ]:
session = boto3.session.Session()

print(session)

In [ ]:
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]

pprint.pprint(region)
pprint.pprint(account_id)

In [100]:
# Agent foundation model name
agent_foundation_model = "anthropic.claude-3-5-sonnet-20240620-v1:0"

In [18]:
# Create bedrock agent IAM policy
agent_bedrock_allow_policy_name = "ucai-bra-iam-policy-name"

# Create IAM policies for agent
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{agent_foundation_model}"
            ]
        }
    ]
}


bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)

In [ ]:
pprint.pprint(agent_bedrock_policy)

In [ ]:
# Create IAM Role for the agent and attach IAM policies
agent_role_name = "ucai-bra-iam-role-name"

assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

In [ ]:
pprint.pprint(agent_role)

In [101]:
# Create bedrock agent 

agent_name = "tbd-bra-name"
agent_description = "tbd-bra-description"
agent_instruction = "You are a weather agent to fetche the current weather in celsius for a given location"

bedrock_agent_client = boto3.client('bedrock-agent')
#bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

create_agent_response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=agent_foundation_model,
    instruction=agent_instruction,
)


In [ ]:
pprint.pprint(create_agent_response)

In [ ]:
agent_id = create_agent_response['agent']['agentId']
print(agent_id)

In [27]:
# Sample weather function 

from datetime import datetime
    
def location_weather_in_c(location_id, fetch_date):
    try:
        # Fetch from Databricks SQL Warehouse based UC function execution 
        return "23"
    except Exception as e:
        raise Exception(f"Error occurred: {e}")

In [28]:
agent_functions = [

    {
        'name': 'location_weather_in_c',
        'description': 'Fetche the current weather in celsius for a given location',
        'parameters': {
            "location_id": {
                "description": "the id of the location",
                "required": True,
                "type": "integer"
            },
            "fetch_date": {
                "description": "the date ",
                "required": True,
                "type": "string"
            }
        },
        'requireConfirmation':'ENABLED'
    },
]

In [ ]:
# Prepare agent group using function schema
agent_action_group_name = "tbd-bda-action-group-name"
agent_action_group_description = "Actions to fetch the weather of a given location for a given date"

agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'customControl': 'RETURN_CONTROL'
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)

response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)

In [ ]:
pprint.pprint(response)

In [106]:
# Invoke agent 
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

# Extract the agentAliasId from the response
agent_alias_id = "****"

## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
enable_trace:bool = False
end_session:bool = False
# Pause to make sure agent alias is ready
# time.sleep(30)

# invoke the agent API
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="What is the weather for location 1234 and date of 2024-11-19",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)



In [ ]:
event_stream = agentResponse['completion']

for event in event_stream:
    if 'returnControl' in event:
        pprint.pp(event)
        print(event["returnControl"]["invocationInputs"][0]["functionInvocationInput"]["function"])

In [ ]:
#Errors 
#EventStreamError: An error occurred (validationException) when calling the InvokeAgent operation: Invocation of model ID anthropic.claude-3-5-haiku-20241022-v1:0 with on-demand throughput isn’t supported. Retry your request with the ID or ARN of an inference profile that contains this model.
# https://us-east-1.console.aws.amazon.com/servicequotas/home/services/bedrock/quotas
## On-demand InvokeModel requests per minute for Anthropic

In [ ]:
# Learnings 
# Enable the model in the AWS account 
# Create the alias and use the alias_id when invoking the agent 


In [ ]:
pprint.pp(event)

In [ ]:
for invocationInput in event["returnControl"]["invocationInputs"]:
    function_to_call = invocationInput["functionInvocationInput"]["function"]
    pprint.pp(function_to_call)
    #TODO 
    # Parse the parameters dynamically  

In [109]:
uc_function_result = "23"

In [ ]:
# On-demand InvokeModel requests per minute for Anthropic Claude 3.5 Sonnet is "1"
# Sleep for 70 seconds to invoke the agent 

time.sleep(70) 

In [110]:
raw_response_with_roc = bedrock_agent_runtime_client.invoke_agent(
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    sessionState={
        'invocationId': event["returnControl"]["invocationId"],
        'returnControlInvocationResults': [{
                'functionResult': {
                    'actionGroup': event["returnControl"]["invocationInputs"][0]["functionInvocationInput"]["actionGroup"],
                    'function': event["returnControl"]["invocationInputs"][0]["functionInvocationInput"]["function"],
                    'confirmationState': 'CONFIRM',
                    'responseBody': {
                        "TEXT": {
                            'body': "weather_in_centigrade: "+str(uc_function_result)
                        }
                    }
                }
        }]}
)

#print(aw_response_with_roc)

In [ ]:
pprint.pp(raw_response_with_roc)

In [ ]:
event_stream3 = raw_response_with_roc['completion']

for event in event_stream3:
    pprint.pp(event)